# **Initialize environment**

In [4]:
#@title Connect to colab runtime and check GPU

#@markdown # Connect to colab runtime and check GPU

#@markdown

#需成功执行一次，

!nvidia-smi
!sudo cp  /home/ec2-user/anaconda3/envs/pytorch_p310/lib/libstdc++.so.6.0.32  /usr/lib64/
!sudo ln -sf /usr/lib64/libstdc++.so.6.0.32  /usr/lib64/libstdc++.so.6

#如果目录存在偏差请执行如下两个命令进行对其查找
# ls -l /usr/lib64/libstdc++.so*
# sudo find / -name libstdc++.so.6*


Wed Nov 29 08:30:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   17C    P8              15W / 300W |      2MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title Clone repository and install requirements

#@markdown # Clone repository and install requirements

#@markdown

#@markdown ### After the execution is completed, the runtime will **automatically restart**

#@markdown

!git clone https://github.com/svc-develop-team/sovits4sagemaker

# %cd /content/sovits4sagemaker
%cd /home/ec2-user/SageMaker/sovits4sagemaker
%pip install --upgrade pip setuptools
%pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118
exit()

In [ ]:
#@title Get pretrained model(Optional but strongly recommend).

#@markdown # Get pretrained model(Optional but strongly recommend).

#@markdown

#@markdown - Pre-trained model files: `G_0.pth` `D_0.pth`
#@markdown   - Place them under logs/44k/ 

#@markdown Get them from svc-develop-team(TBD) or anywhere else.

#@markdown Although the pretrained model generally does not cause any copyright problems, please pay attention to it. For example, ask the author in advance, or the author has indicated the feasible use in the description clearly.

download_pretrained_model = True #@param {type:"boolean"}
D_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_D_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_D_320000.pth"] {allow-input: true}
G_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth" #@param ["https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth", "https://huggingface.co/1asbgdh/sovits4.0-volemb-vec768/resolve/main/clean_G_320000.pth", "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/vol_emb/clean_G_320000.pth"] {allow-input: true}

download_pretrained_diffusion_model = True #@param {type:"boolean"}
diff_model_URL = "https://huggingface.co/datasets/ms903/Diff-SVC-refactor-pre-trained-model/resolve/main/fix_pitch_add_vctk_600k/model_0.pt" #@param {type:"string"}

%cd /home/ec2-user/SageMaker/sovits4sagemaker

if download_pretrained_model:
    !curl -L {D_0_URL} -o logs/44k/D_0.pth
    !md5sum logs/44k/D_0.pth
    !curl -L {G_0_URL} -o logs/44k/G_0.pth
    !md5sum logs/44k/G_0.pth

if download_pretrained_diffusion_model:
    !mkdir -p logs/44k/diffusion
    !curl -L {diff_model_URL} -o logs/44k/diffusion/model_0.pt
    !md5sum logs/44k/diffusion/model_0.pt

# **Dataset preprocessing**

目的是把需要训练的声音放到(dataset_raw/)目录中

文件结构如下: 这里一定要注意speaker/speaker0等是重要标识

```
YourforlderforSingleSpeakers
└───speaker
    ├───xxx1-xxx1.wav
    ├───...
    └───Lxx-0xx8.wav
```

```
YourFolderforMultipleSpeakers
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav
```

**Even if there is only one speaker, a folder named `{speaker_name}` is needed.**

In [ ]:
# 克隆slide tools
%cd /home/ec2-user/SageMaker
%git clone https://github.com/openvpi/audio-slicer

In [ ]:
#更新必要的音频处理相关的库
%pip install numpy
%pip install librosa
%pip install soundfile

In [ ]:
%mkdir /home/ec2-user/SageMaker/audio-slicer/audiowav

#运行后将wav文件拷贝到audio-slicer/audiowav目录下

In [ ]:
%cd /home/ec2-user/SageMaker/audio-slicer/
SPEAKER_NAME = "speak1" #需要克隆的人的标志
import os
if not os.path.exists("/home/ec2-user/SageMaker/audio-slicer/dataset_raw"):
    %mkdir /home/ec2-user/SageMaker/audio-slicer/dataset_raw
if not os.path.exists("/home/ec2-user/SageMaker/audio-slicer/dataset_raw/"+SPEAKER_NAME):
    %mkdir /home/ec2-user/SageMaker/audio-slicer/dataset_raw/{SPEAKER_NAME}


import librosa  # Optional. Use any library you like to read audio files.
import soundfile  # Optional. Use any library you like to write audio files.

from slicer2 import Slicer

audio, sr = librosa.load('audiowav/example.wav', sr=None, mono=False)  # Load an audio file with librosa. 
slicer = Slicer(
    sr=sr,
    threshold=-40,
    min_length=5000,
    min_interval=300,
    hop_size=10,
    max_sil_kept=500
)
chunks = slicer.slice(audio)
for i, chunk in enumerate(chunks):
    if len(chunk.shape) > 1:
        chunk = chunk.T  # Swap axes if the audio is stereo.
    soundfile.write(f'/home/ec2-user/SageMaker/audio-slicer/dataset_raw/{SPEAKER_NAME}/{i}{SPEAKER_NAME}_{i}.wav', chunk, sr)

In [ ]:
#处理的音频拷贝到训练目录下
%cp -vr /home/ec2-user/SageMaker/audio-slicer/dataset_raw /home/ec2-user/SageMaker/sovits4sagemaker/

In [ ]:
#@title Resample to 44100Hz and mono

#@markdown # Resample to 44100Hz and mono

#@markdown

%cd /home/ec2-user/SageMaker/sovits4sagemaker
!python resample.py

In [ ]:
#@title Train cluster model (Optional)

#@markdown # Train cluster model (Optional)

#@markdown #### Details see [README.md#cluster-based-timbre-leakage-control](https://github.com/svc-develop-team/sovits4sagemaker#cluster-based-timbre-leakage-control)

#@markdown

%cd /home/ec2-user/SageMaker/sovits4sagemaker
!python cluster/train_cluster.py --gpu

In [ ]:
#@title Divide filelists and generate config.json

#@markdown # Divide filelists and generate config.json

#@markdown

%cd /home/ec2-user/SageMaker/sovits4sagemaker
%mkdir -p /home/ec2-user/SageMaker/sovits4sagemaker/dataset/44k

speech_encoder = "vec768l12" #@param ["vec768l12", "vec256l9", "hubertsoft", "whisper-ppg", "whisper-ppg-large"]
use_vol_aug = False #@param {type:"boolean"}
vol_aug = "--vol_aug" if use_vol_aug else ""

from pretrain.meta import download_dict
download_dict = download_dict()

url = download_dict[speech_encoder]["url"]
output = download_dict[speech_encoder]["output"]

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}
  !md5sum {output}

!python preprocess_flist_config.py --speech_encoder={speech_encoder} {vol_aug}

# **Trainning**

In [ ]:
#@title Train index model (Optional)

#@markdown # Train index model (Optional)

#@markdown #### Details see [README.md#feature-retrieval](https://github.com/svc-develop-team/sovits4sagemaker#feature-retrieval)

#@markdown

%cd /home/ec2-user/SageMaker/sovits4sagemaker
!python train_index.py -c configs/config.json

In [ ]:
#@title Generate hubert and f0

#@markdown # Generate hubert and f0

#@markdown
%cd /home/ec2-user/SageMaker/sovits4sagemaker

f0_predictor = "crepe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]
use_diff = True #@param {type:"boolean"}

import os
if f0_predictor == "rmvpe" and not os.path.exists("./pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("./pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt


diff_param = ""
if use_diff:
  diff_param = "--use_diff"

  if not os.path.exists("./pretrain/nsf_hifigan/model"):
    !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
    !md5sum nsf_hifigan_20221211.zip
    !unzip nsf_hifigan_20221211.zip
    !rm -rf pretrain/nsf_hifigan
    !mv -v nsf_hifigan pretrain

!python preprocess_hubert_f0.py --f0_predictor={f0_predictor} {diff_param}

In [ ]:
#@title Start training

#@markdown # Start training

#@markdown If you want to use pre-trained models, upload them to logs/44K下

#@markdown

%cd /home/ec2-user/SageMaker/sovits4sagemaker

#@markdown Whether to enable tensorboard
tensorboard_on = False  #@param {type:"boolean"}

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

config_path = "/home/ec2-user/SageMaker/sovits4sagemaker/configs/config.json"

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}

!python train.py -c {config_path} -m 44k

In [ ]:
#@title Train diffusion model (Optional)

#@markdown # Train diffusion model (Optional)

#@markdown #### Details see [README.md#-about-shallow-diffusion](https://github.com/svc-develop-team/sovits4sagemaker#-about-shallow-diffusion)

#@markdown

%cd /home/ec2-user/SageMaker/sovits4sagemaker

import os
if not os.path.exists("./pretrain/nsf_hifigan/model"):
  !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
  !unzip nsf_hifigan_20221211.zip
  !rm -rf pretrain/nsf_hifigan
  !mv -v nsf_hifigan pretrain

#@markdown Whether to enable tensorboard
tensorboard_on = False  #@param {type:"boolean"}

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train_diff.py -c configs/diffusion.yaml

# **多国语音**
一共支持32种语言，具体使用那种语言请查阅

https://docs.aws.amazon.com/polly/latest/dg/ntts-voices-main.html


In [ ]:
#使用polly可以产生36个国家语言，因为这里使用声纹克隆所以推荐的语音角色如下

import os
import time
import json
import boto3
from contextlib import closing
import tempfile
import urllib3
from botocore.exceptions import ClientError
import json
import decimal
text='مرحبا بالجميع ، أنا مساعد صوت Amazon Web Services يمكنه قراءة معظم لغات ولهجات العالم ، ويمكنه محاكاة بصمة صوت شخص معين'
languageCode = 'ar-AE' #语种
voiceId = 'Hala' #性别角色
speed_prefix = '<prosody rate="fast">' #变速
ssmltext='<speak>'+speed_prefix+text+'</prosody></speak>'
c = boto3.client('polly',region_name='us-east-1')

    
try:
    response = c.synthesize_speech(Engine='neural',
                         LanguageCode=languageCode, 
                         OutputFormat='mp3',
                         Text=ssmltext, 
                         TextType='ssml',
                         VoiceId=voiceId)
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)
    
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
           output = os.path.join("/home/ec2-user/SageMaker/sovits4sagemaker/raw", "speech.mp3")
           try:
            # Open a file for writing the output as a binary stream
                with open(output, "wb") as file:
                   file.write(stream.read())
           except IOError as error:
              # Could not write to file, exit gracefully
              print(error)
              sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")
    sys.exit(-1)


    



# **Inference**
### Upload wav files from this notebook


In [ ]:
#此步只需成功执行一次
#title Download nsf_hifigan if you need it

%cd /home/ec2-user/SageMaker/sovits4sagemaker
!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /home/ec2-user/SageMaker/sovits4sagemaker/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain


In [ ]:
#此步骤执行比较慢只需成功执行一次
%conda install ffmpeg
%conda install ffmpy
%conda install ffprobe
%conda install pydub 

In [ ]:
#如果初始声音为mp3则执行此步骤

%cd /home/ec2-user/SageMaker/sovits4sagemaker/raw


from pydub import AudioSegment
import wave

# 读取mp3的波形数据
sound = AudioSegment.from_file("/home/ec2-user/SageMaker/sovits4sagemaker/raw/speech.mp3", format = 'MP3')

f = wave.open("YourWAVFile.wav", 'wb')
f.setnchannels(1)   # 频道数
f.setsampwidth(2)   # 量化位数
f.setframerate(16000)   # 取样频率
f.setnframes(len(sound._data))   # 取样点数，波形数据的长度
f.writeframes(sound._data)   # 写入波形数据
f.close()

In [ ]:
#@title Start inference (and download)

#@markdown # Start inference (and download)

#@markdown Parameters see [README.MD#Inference](https://github.com/svc-develop-team/sovits4sagemaker#-inference)

#@markdown

wav_filename = "YourWAVFile.wav"  #@param {type:"string"}
model_filename = "G_800.pth"  #@param {type:"string"}
model_path = "/home/ec2-user/SageMaker/sovits4sagemaker/logs/44k/" + model_filename
speaker = "speak1"  #@param {type:"string"}
trans = "0"  #@param {type:"string"}
cluster_infer_ratio = "0"  #@param {type:"string"}
auto_predict_f0 = True  #@param {type:"boolean"}
apf = ""
if auto_predict_f0:
  apf = " -a "

f0_predictor = "crepe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]

enhance = False  #@param {type:"boolean"}
ehc = ""
if enhance:
  ehc = " -eh "
#@markdown

#@markdown Generally keep default:
config_filename = "config.json"  #@param {type:"string"}
config_path = "/home/ec2-user/SageMaker/sovits4sagemaker/configs/" + config_filename

# from pretrain.meta import get_speech_encoder
# url, output = get_speech_encoder(config_path)

import os

if f0_predictor == "rmvpe" and not os.path.exists("/home/ec2-user/SageMaker/sovits4sagemaker/pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("/home/ec2-user/SageMaker/sovits4sagemaker/pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt

# if not os.path.exists(output):
#   !curl -L {url} -o {output}

kmeans_filenname = "kmeans_10000.pt"  #@param {type:"string"}
kmeans_path = "/home/ec2-user/SageMaker/sovits4sagemaker/logs/44k/" + kmeans_filenname
slice_db = "-40"  #@param {type:"string"}
wav_format = "wav"  #@param {type:"string"}

key = "auto" if auto_predict_f0 else f"{trans}key"
cluster_name = "" if cluster_infer_ratio == "0" else f"_{cluster_infer_ratio}"
isdiffusion = "sovits"
wav_output = f"/home/ec2-user/SageMaker/sovits4sagemaker/results/{wav_filename}_{key}_{speaker}{cluster_name}_{isdiffusion}_{f0_predictor}.{wav_format}"

%cd /home/ec2-user/SageMaker/sovits4sagemaker
!python inference_main.py -n {wav_filename} -m {model_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -c {config_path} -cm {kmeans_path} -sd {slice_db} -wf {wav_format} {apf} --f0_predictor={f0_predictor} {ehc}

#@markdown

